In [49]:
# Version 1.01
# Authors: 1) Hamza Nadeem
#          2) Monica Restrepo
#          3) Nency Borad
#          4) Ajinkya Mukherjee
# Date      Name  Version Description
# 5/6/2022  SCL   1.01    Refactor code for Hypothesis (part II), and remove all
#                         EDA code.
#                 1.01    Perform Feature Selection and create utility matrix
#                         labelled data, for supervised machine learning
#
# -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -

In [50]:
# Features: 1) PopSize 
#           2) Time
#           3) Date
#           4) Num of complaints
#           5) Type of Crime [Felony, Misdorminor, Violation]
#           -) Class = [ Low Crime, Medium Crime, High Crime ]
#               10 = Low
#              200 = Medium
#             9000 = High

# Hypo: FIgure out which zip codes will increase/dec in crime rate? 
#and we need to categorize the dataset into LOW MID HIGH  
# Algorithms: 1) SVM
#             2) Bayes (NB)
#             3) Random Forest
#             4) Logistic Regression (0 or 1)
# -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -

In [51]:
# Questions to ask Professor
# Q1: We did all this, how do add them to our Feature list
# Q2: How do we Classify what is high,low,medium ( How do we get the score) = Range of the score
# Q3: How do we get Population by Zipcode from total population of Burrow
# Q4: Can we use the Number of Crimes in Each Zipcode as our Feature for number of complaints / per area
# Q5: How do we add Labels to our dataset inorder to Train them - adding Labels to the Testing set
# Q6: How does the iris dataset have so many features & Classes, how do we optimize our features accordingly 
#     and why do we require so many featueres and Classes?
# Q7: Are we implementing the Alogirthms correct

In [52]:
#from google.colab import drive
#drive.flush_and_unmount()
#drive.mount('/content/gdrive');

In [53]:
# !pip install modin[ray]
!pip install pgeocode

In [54]:
pip install scikit-learn-intelex

Note: you may need to restart the kernel to use updated packages.Collecting daal==2021.2.3
  Using cached daal-2021.2.3-py2.py3-none-win_amd64.whl (63.9 MB)
  Using cached tbb-2021.6.0-py3-none-win_amd64.whl (278 kB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2



ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [55]:
from sklearnex import patch_sklearn 

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [56]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


df1 = pd.read_csv(r'C:\Users\hnade\Downloads\AI\final_project_csvs\NYPD_Complaint_Data_Historic.csv', low_memory=False)
df2 = pd.read_csv(r'C:\Users\hnade\Downloads\AI\final_project_csvs\Crime_Column_Description.csv', low_memory=False)
df3 = pd.read_csv(r'C:\Users\hnade\Downloads\AI\final_project_csvs\Population_by_Borough_NYC.csv', low_memory=False)
#df4 = pd.read_csv('/content/gdrive/MyDrive/AI/final_project_csvs/ziptoborough.csv')
# newData = pd.read_csv('US.csv')
# zip_codes = pd.read_csv('/content/gdrive/MyDrive/AI/final_project_csvs/us.csv')

# zip_codes


In [57]:

df5 = pd.DataFrame(df1[['Longitude','Latitude']])
# df1['Latitude']
df5

,Longitude,Latitude
0,-73.916661,40.828848
1,-73.784557,40.697338
2,-73.945052,40.802607
3,-73.726339,40.654549
4,-73.987891,40.738002
...,...,...
1048570,-73.920767,40.806932
1048571,-73.895227,40.660901
1048572,-74.004681,40.723909
1048573,-73.872939,40.877554


In [58]:
# 54.143 - Lat
# -165.7854 - Long
# zc = zip_codes.head(10)
# zc


#helper

# def findZip(lat, long):
  # print('lat', "%.4f" % lat)

  # lat = "%.4f" % lat
  # long = "%.4f" % long
  # print('lat', lat)
  # print('long', long)
  # t = np.where((zip_codes['Latitude'] == lat) & (zip_codes['Longitude'] == long), zip_codes['ZIP'], 99999)
  # print('here====>', t)
  # if zip_codes['Latitude'] == lat:
  #   if zip_codes['Longitude'] == long:
  #     print('We got it!!', zip_codes['Zip'])

# s= df5.head(20)
# df5['Longitude'] = df5['Longitude'].astype(float)
# df5['Latitude'] = df5['Latitude'].astype(float)
# df5['Longitude'] = "%.6f" % df5['Longitude']
# df5['Latitude'] = "%.6f" % df5['Latitude']

# df5 = np.where(df1['Latitude'] == zc['Latitude'] and df1['Longitude'] == zc['Longitude'],zc['zip'],99999)
# # t
# for i, value in s.iterrows():
#   # print('idx', i)
#   # print('value ====> ', value)
#   findZip(value['Latitude'], value['Longitude'])
    # print('lat only :::::>', value['Latitude'])


# df5['Zipcode'] = np.where(df1['Latitude'] == zip_codes['Latitude'] and df1['Longitude'] == zip_codes['Longitude'],zip_codes['zip'],99999)

# newData['LONG']
# zipcode_Dict = newData.set_index('ZIP').T.to_dict('list')

In [59]:
# zipcode_Dict

In [60]:
def makeCategories(df):
  category = [health,residence,worship,pub,retail,Tranportation,Food_Beverage,finance,other]
  residence_count = 0
  health_count = 0
  worship_count = 0
  pub_count = 0
  retial_count = 0
  tranport_count = 0
  foodbev_count = 0
  finance_count = 0
  other_count = 0
  df['Food & beverage'] = 0

  for a in df['PREM_TYP_DESC']:
    if a in residence:      
      residence_count += 1
      
    if a in other:
      other_count += 1

    if a in finance:
      finance_count += 1

    if a in Food_Beverage:
      foodbev_count += 1
      df['Food and beverage'] = 1
      pass

    if a in Tranportation:
      tranport_count += 1

    if a in retail:
      retial_count += 1

    if a in worship:
      worship_count += 1
        
    if a in health:
      health_count += 1
      
  print('residence',residence_count)
  print('other',other_count)
  print('finance',finance_count)
  print('food',foodbev_count)        
  print('Transport',tranport_count)
  print('Retail',retial_count)
  print('Worship',worship_count)
  print('Health',health_count)

  return residence_count, other_count, finance_count, foodbev_count, tranport_count, retial_count, worship_count, health_count, pub_count

In [61]:
#df1 = df1.sample(500000)

In [62]:
health = ['DRUG STORE','DOCTOR/DENTIST OFFICE','HOSPITAL']
residence = ['RESIDENCE-HOUSE','RESIDENCE - APT. HOUSE','RESIDENCE - PUBLIC HOUSING']
worship = ['CHURCH','SYNAGOGUE','OTHER HOUSE OF WORSHIP','MOSQUE']
pub = ['STREET','PUBLIC BUILDING','PARK/PLAYGROUND','PARKING LOT/GARAGE (PUBLIC)','PRIVATE/PAROCHIAL SCHOOL','BOOK/CARD','MARINA/PIER',
       'PUBLIC SCHOOL','CEMETERY']
retail = ['DRY CLEANER/LAUNDRY','GAS STATION','CHAIN STORE','BEAUTY & NAIL SALON',
          'COMMERCIAL BUILDING','HOTEL/MOTEL','SMALL MERCHANT','PARKING LOT/GARAGE (PRIVATE)',
          'DEPARTMENT STORE','GYM/FITNESS FACILITY','VARIETY STORE','CLOTHING/BOUTIQUE','TELECOMM. STORE',
          'JEWELRY','CANDY STORE','STORAGE FACILITY','VIDEO STORE']
Tranportation = ['TUNNEL','TRANSIT - NYC SUBWAY','BUS (NYC TRANSIT)','FERRY/FERRY TERMINAL','BUS STOP','TAXI (LIVERY LICENSED)',
                 'BRIDGE','AIRPORT TERMINAL','HIGHWAY/PARKWAY','BUS (OTHER)',
                 'TRANSIT FACILITY (OTHER)','BUS TERMINAL','TAXI (YELLOW LICENSED)','TAXI/LIVERY (UNLICENSED)','TRAMWAY']
Food_Beverage= ['BAR/NIGHT CLUB' , 'FAST FOOD' , 'RESTAURANT/DINER' , 'GROCERY/BODEGA' , 'FOOD SUPERMARKET' , 'SOCIAL CLUB/POLICY' , 'LIQUOR STORE']
finance = ['CHECK CASHING BUSINESS','BANK','ATM','LOAN COMPANY']
other = ['OTHER','STORE UNCLASSIFIED','CONSTRUCTION SITE','FACTORY/WAREHOUSE','OPEN AREAS (OPEN LOTS)','ABANDONED BUILDING','SHOE','PHOTO/COPY']

In [63]:
df1.head(4)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,44.0,INSIDE,BAR/NIGHT CLUB,NaN,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,103.0,OUTSIDE,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,105.0,INSIDE,RESIDENCE-HOUSE,NaN,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)"


In [64]:
#residencecount, othercount, financecount, foodbevcount, tranportcount, retialcount, worshipcount, healthcount, pubcount = makeCategories(df1)

In [65]:
# #Assign zipcodes to each row

# import geopy
# from geopy.point import Point

# def get_zipcode(df5, geolocator, Latitude, Longitude):
#   zipcode = 99999
#   try: 
#     # location = geolocator.reverse(Point(df1["Latitude"], df1["Longitude"]))
#     location = geolocator.reverse(Point(df5["Latitude"], df5["Longitude"]))
#     zipcode = location.raw['address']['postcode']
#     zipcode = int(zipcode)
#   except:
#     pass
#   return zipcode

# geolocator = geopy.Nominatim(user_agent='AI-application')
# df1['Zipcodes'] = df1.apply(get_zipcode, axis=1, geolocator=geolocator, Latitude=df1["Latitude"], Longitude=df1["Longitude"])


In [66]:
# from multiprocessing import Process
# p1 = Process(target=get_zipcode, args=(df5,))
# df1.head()

HYPOTHESIS: Segregate Burrows based on type of crime. 

In [67]:
# Pop Size
feature_1 = df3[['Borough','2010','2020']]
feature_1 = feature_1.apply(lambda x: x.str.upper() if x.dtype == "object" else x)  
feature_1.rename(columns = {'Borough':'BORO_NM'}, inplace = True)
feature_1


,BORO_NM,2010,2020
0,NYC TOTAL,"8,242,624","8,550,971"
1,BRONX,"1,385,108","1,446,788"
2,BROOKLYN,"2,552,911","2,648,452"
3,MANHATTAN,"1,585,873","1,638,281"
4,QUEENS,"2,250,002","2,330,295"
5,STATEN ISLAND,"468,730","487,155"


In [68]:
#df1 = df1.dropna()
#df1.head(100)
null_columns=df1.columns[df1.isnull().any()]
df1[null_columns].isnull().sum()
df1.drop(329156, inplace=True)
print(df1[df1['ADDR_PCT_CD'].isnull()][null_columns].head())

Empty DataFrame
Columns: [CMPLNT_FR_DT, CMPLNT_TO_DT, CMPLNT_TO_TM, OFNS_DESC, PD_CD, PD_DESC, CRM_ATPT_CPTD_CD, ADDR_PCT_CD, LOC_OF_OCCUR_DESC, PREM_TYP_DESC, PARKS_NM, HADEVELOPT, X_COORD_CD, Y_COORD_CD, Latitude, Longitude, Lat_Lon]
Index: []


In [69]:
df1['ADDR_PCT_CD'].unique()
# df1['ADDR_PCT_CD']

array([ 44., 103.,  28., 105.,  13.,  71.,   7.,  46.,  48.,  19.,  41.,
        14.,  67.,  17.,  61., 102., 110., 108.,  75.,  73.,  60.,  68.,
        79., 121.,  23.,  42., 115.,  52., 122.,   1.,  72., 109.,  24.,
        81.,  90., 112.,  43.,  84.,  47.,  77., 101.,  83., 113., 120.,
        70.,  69.,  66., 114.,  76.,  63.,  45., 106.,  10.,  78.,   6.,
         5.,  94.,  40.,  34.,  32.,  50.,  25., 100.,  18.,  20., 111.,
       107.,  30.,  49.,  88.,  26., 123.,   9., 104.,  33.,  62.,  22.])

In [70]:
# Time of Day
feature_set = df1[['CMPLNT_FR_TM','BORO_NM','ADDR_PCT_CD']]
feature_set.set_index('ADDR_PCT_CD', inplace=True)


In [71]:
bronx_ = 1385108
brook_ = 2552911
manha_ = 1585873
queen_ = 2250002
state_ = 468730

In [72]:
feature_set['Count'] = df1.groupby(by='ADDR_PCT_CD')['CMPLNT_NUM'].count()
feature_set['Pop'] = 0
feature_set.loc[feature_set["BORO_NM"] == "MANHATTAN", "Pop"] = manha_
feature_set.loc[feature_set["BORO_NM"] == "BRONX", "Pop"] = bronx_
feature_set.loc[feature_set["BORO_NM"] == "QUEENS", "Pop"] = queen_
feature_set.loc[feature_set["BORO_NM"] == "STATEN ISLAND", "Pop"] = state_
feature_set.loc[feature_set["BORO_NM"] == "BROOKLYN", "Pop"] = brook_


C:\Users\hnade\AppData\Local\Temp/ipykernel_4352/2546497666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_set['Count'] = df1.groupby(by='ADDR_PCT_CD')['CMPLNT_NUM'].count()
C:\Users\hnade\AppData\Local\Temp/ipykernel_4352/2546497666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_set['Pop'] = 0
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [73]:
#-------------------------------------------------------#
# Start Part 2 of the Prject using Keras/scikitlearn

# sklearn imports below
from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time



In [74]:
print("minimum crime count => ", feature_set['Count'].min())
print("max crime count => ", feature_set['Count'].max())
print("mean crime count => ", feature_set['Count'].mean())

minimum crime count =>  832
max crime count =>  36460
mean crime count =>  15972.79769858875


In [ ]:
# Add Class Low Medium, High
# if time between 6pm and 4am, and count > 10, then high_crime 26217
# if time between 6pm and 4am, and count < 10 but > 5, then medium_crime 7570
# if time between 6pm and 4am, and count < 5, then low_crime
#
#feature_set['Class'] = 0

# Change Count everytime you change Sample Size
feature_set.loc[feature_set["Count"] >= 26217, "Class"]          = 'high'
feature_set.loc[feature_set["Count"] <= 7570, "Class"]           = 'med'
feature_set.loc[feature_set["Count"].between(7571,26216,inclusive="both"), "Class"] = 'med'

 
# Split time into hh mm ss
feature_set[['hour', 'minutes', 'seconds']] = feature_set['CMPLNT_FR_TM'].str.split(':', expand=True)
feature_set.head()

In [ ]:
# Drop the Boro Name for now
#feature_set.drop(['BORO_NM'], axis=1, inplace=True)

# Can we add tags to each data based on location of crime ( Health, residence, Retail)

In [ ]:
# Creating the Main Function/ Calling these Algorithms
X = feature_set[['Pop','hour','Count']]
y = feature_set[['Class']]

In [ ]:
# Function to Create 
def mlSVM(X,y,p): # Ajinkya
  svm_prediction = 0
  #svcObj = LinearSVC()
  svcObj = SVC()
  start = time.time()
  svcObj.fit(X,y)
  stop = time.time()
  svm_prediction = svcObj.predict(p)
  print(f"Training time: {stop - start}s")
    
  return svm_prediction

def mlRegression(X, y, p): # Nency
  # init y_predict
  linear_prediction = 0
  
  regObj = LogisticRegression()
  start = time.time()
  regObj.fit(X, y)
  stop = time.time()
  logistic_prediction = regObj.predict(p)
  logistic_prediction_score = regObj.score(X,y)
  print(f"Training time: {stop - start}s")
  return logistic_prediction,logistic_prediction_score

def mlRandomForest(X, y, p): # Hamza
  # init y_predict
  rf_prediction = 0

  rfObj = RandomForestClassifier()
  start = time.time()
  rfObj.fit(X, y)
  stop = time.time()
  rf_prediction = rfObj.predict(p)
  print(f"Training time: {stop - start}s")
  return rf_prediction

def mlBayes(X, y, p): # Monica
  # init y_predict
  bayes_prediction = 0

  bayesObj = GaussianNB()   # create an instance of the object
  start = time.time()
  bayesObj.fit(X,y)        # fit or train the model
  stop = time.time()
  bayes_prediction = bayesObj.predict(p)
  print(f"Training time: {stop - start}s")
  return bayes_prediction

  


In [ ]:
feature_set

In [ ]:
feature_set.isnull().sum()

In [ ]:
X_train , X_test , y_train, y_test = train_test_split(X, y, test_size=0.3,random_state= 42)
X_train= X_train.astype(int)
X_test = X_test.astype(int)

# y_train = y_train.astype(int)
# y_test = y_test.astype(int)



y_reg,score_regression = mlRegression(X_train, y_train, X_test)
score_regression = accuracy_score(y_test,y_reg)

y_rf  = mlRandomForest(X_train, y_train, X_test)
score_randomForest = accuracy_score(y_test,y_rf)

y_bayes = mlBayes(X_train, y_train, X_test)
score_bayes = accuracy_score(y_test,y_bayes)





print('The accuracy score is bayes = ', score_bayes)

print('The accuracy score is reg   = ', score_regression)
print('The accuracy score is rfores= ', score_randomForest)




Plot predicted values  <br> y= test x= Zipcode <br> Heat Map If Possible otherwise any chart that can help us visualize the predicted values<br>

In [ ]:
y_svm = mlSVM(X_train, y_train, X_test)
score_SVM = accuracy_score(y_test,y_svm)
print('The accuracy score is svm   = ', score_SVM)

In [ ]:
# Crime Count per precinct
# min = 832
# mean = 15973
# max = 36460

# How did we calculate our Class range for low med high 
# crime rate
# First Trial
# For Crime Rate Class Low: (mean - min)/2 : (15973 - 832)/2 = 7570
# For Crime Rate Class High: (max - mean): 36460 - 15973 : 26217
# For Crime Rate Class Med: everything in between  7571 - 26216
# Accuracy Score:
#                 bayes:0.9991512290489474
#                 svm:
#                 reg: 0.9254750494719023
#                 rfores: 1.0 

In [ ]:
len(X_test)

In [ ]:
import pandas as pd
dfgraph = pd.DataFrame({'Algorithms':['Bayes', 'Regression', 'Random Forest', 'SVM'], "Accuracy":[score_bayes, score_regression,score_randomForest, score_SVM]})
ax = dfgraph.plot.bar(x='Algorithms', y='Accuracy', rot=0)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import make_blobs
X, y = make_blobs(330, 3, centers=3, random_state=42, cluster_std=1.5)

np.save('./datasv.npy', (X_train, X_test, y_train, y_test))
X_train, X_test, y_train, y_test = np.load('./datasv.npy', allow_pickle=True)

plt.scatter(y_test, y_test, c=y, s=50, cmap='RdBu');

In [ ]:
from sklearn.svm import SVC
import seaborn as sns
sns.set()
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
inputs,targets=make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.7)
plt.scatter(X_train[:,0],y_test[:,1],c=targets, cmap='winter')
plt.title('Data points of the 2 classes')
plt.show()

In [ ]:
from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix

# Configuration options
blobs_random_seed = 42
centers = [(0,0), (5,5)]
cluster_std = 1.5
frac_test_split = 0.33
num_features_for_samples = 2
num_samples_total = 1000

# Generate data
inputs, targets = make_blobs(n_samples = num_samples_total, centers = centers, n_features = num_features_for_samples, cluster_std = cluster_std)
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=frac_test_split, random_state=blobs_random_seed)

# Save and load temporarily
np.save('./datasv.npy', (X_train, X_test, y_train, y_test))
X_train, X_test, y_train, y_test = np.load('./datasv.npy', allow_pickle=True)

# Generate scatter plot for training data 
plt.scatter(X_train[:,0], X_train[:,1])
plt.title('Linearly separable data')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

# Initialize SVM classifier
clf = svm.SVC(kernel='linear')

# Fit data
clf = clf.fit(X_train, y_train)

In [ ]:
plt.scatter(y_train.iloc[:0,:1],y_train.iloc[:0,:1])
plt.scatter(y_test.iloc[:,0], y_test.iloc[:,1], color='red')
plt.title('Linearly separable data with support vectors')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show

In [ ]:
#Plot SVM

from sklearn import svm, datasets


def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

model = svm.SVC(kernel='linear')
clf = model.fit(X, y)

fig, ax = plt.subplots()
# title for the plots
title = ('Decision surface of linear SVC ')
# Set-up grid for plotting.
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)


ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('y label here')
ax.set_xlabel('x label here')
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(title)
ax.legend()
plt.show()